In [24]:
import glob
import zipfile
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
!unzip -o "/content/drive/MyDrive/data/Arquivos CSV.zip" -d "/content/drive/MyDrive/data"

Archive:  /content/drive/MyDrive/data/Arquivos CSV.zip
  inflating: /content/drive/MyDrive/data/Arquivos CSV/2016.csv  
  inflating: /content/drive/MyDrive/data/Arquivos CSV/2017.csv  
  inflating: /content/drive/MyDrive/data/Arquivos CSV/2018.csv  
  inflating: /content/drive/MyDrive/data/Arquivos CSV/2019.csv  
  inflating: /content/drive/MyDrive/data/Arquivos CSV/2020.csv  
  inflating: /content/drive/MyDrive/data/Arquivos CSV/2021.csv  
  inflating: /content/drive/MyDrive/data/Arquivos CSV/2022.csv  
  inflating: /content/drive/MyDrive/data/Arquivos CSV/2023.csv  


In [26]:
arquivos_csv = glob.glob('/content/drive/MyDrive/data/Arquivos CSV/*.csv')
lista_arquivos = ["/content/drive/MyDrive/data/Arquivos CSV/" + arquivo.split('/')[-1] for arquivo in arquivos_csv]
lista_arquivos.sort()

## Filtragem dos CSVs

In [27]:
with open('/content/drive/MyDrive/data/Arquivos CSV/Impostos.txt', 'r') as arquivo:
    impostos = arquivo.read().splitlines()

In [28]:
#@markdown Adicionando Novas Cidades
arquivo_cidades = '/content/drive/MyDrive/data/Arquivos CSV/Cidades.txt'

# Função para carregar a lista de cidades do arquivo
def carregar_cidades():
    try:
        with open(arquivo_cidades, 'r') as arquivo:
            return arquivo.read().splitlines()
    except FileNotFoundError:
        return []

# Função para salvar a lista de cidades no arquivo
def salvar_cidades(cidades):
    with open(arquivo_cidades, 'w') as arquivo:
        for cidade in cidades:
            arquivo.write(cidade + '\n')

# Carrega a lista inicial de cidades
cidades = carregar_cidades()

# Criação dos widgets
texto_cidade = widgets.Text(
    description="Cidade:",
    placeholder="Digite o nome da Cidade (ex: Nome da Cidade - UF)",  # Coloca o texto como placeholder
    layout=widgets.Layout(width='500px')  # Aumenta a largura do campo de texto
)
botao_adicionar = widgets.Button(description="Adicionar Cidade")
output = widgets.Output()

# Função chamada ao clicar no botão
def on_button_clicked(b):
    nova_cidade = texto_cidade.value.strip()  # Obtém o texto do campo
    if nova_cidade and nova_cidade not in cidades:  # Verifica se o campo não está vazio e evita duplicatas
        cidades.append(nova_cidade)  # Adiciona a nova cidade à lista
        salvar_cidades(cidades)  # Salva a lista atualizada no arquivo
        with output:
            output.clear_output()  # Limpa o output anterior
            print(f"Cidade '{nova_cidade}' adicionada com sucesso!")
            print(f"Lista de Cidades: '{cidades}'")
    elif nova_cidade in cidades:
        with output:
            output.clear_output()
            print(f"A cidade '{nova_cidade}' já está na lista.")
            print(f"Lista de Cidades: '{cidades}'")
    else:
        with output:
            output.clear_output()
            print("Digite um nome de cidade válido.")
    texto_cidade.value = ""  # Limpa o campo de texto após a adição

# Configura o evento de clique do botão
botao_adicionar.on_click(on_button_clicked)

# Exibe os widgets
display(texto_cidade, botao_adicionar, output)

Text(value='', description='Cidade:', layout=Layout(width='500px'), placeholder='Digite o nome da Cidade (ex: …

Button(description='Adicionar Cidade', style=ButtonStyle())

Output()

In [29]:
for arquivo in lista_arquivos:
    df = pd.read_csv(arquivo, sep=';', encoding='latin1', header=5)
    df['Valor'] = df['Valor'].str.replace(',', '.').astype(float)
    filtro_cidades = df['Instituição'].str.contains('|'.join(cidades), case=False, na=False)
    filtro_impostos = df['Conta'].str.contains('|'.join(impostos), case=False, na=False)
    df_filtrado = df[filtro_cidades & filtro_impostos]
    df_filtrado.to_csv(arquivo, index=False)

## Concatenação dos CSVs

In [30]:
lista_dfs = []
ano = 2016

for arquivo in lista_arquivos:
    df = pd.read_csv(arquivo)
    df['Ano'] = ano
    ano += 1
    lista_dfs.append(df)

In [31]:
df_geral = pd.concat(lista_dfs, ignore_index=True)
df_geral.drop(columns=['Identificador da Conta'], inplace=True)
df_geral['Coluna'] = df_geral['Coluna'].astype(str)

df_mensal = df_geral.loc[~df_geral['Coluna'].str.contains("PREVISÃO|TOTAL")].copy()

In [32]:
mes_map = {
    '<MR>': 12,      # Dezembro
    '<MR-1>': 11,    # Novembro
    '<MR-2>': 10,    # Outubro
    '<MR-3>': 9,     # Setembro
    '<MR-4>': 8,     # Agosto
    '<MR-5>': 7,     # Julho
    '<MR-6>': 6,     # Junho
    '<MR-7>': 5,     # Maio
    '<MR-8>': 4,     # Abril
    '<MR-9>': 3,     # Março
    '<MR-10>': 2,    # Fevereiro
    '<MR-11>': 1     # Janeiro
}
df_mensal['Coluna'] = df_mensal['Coluna'].map(mes_map)
df_mensal['Coluna'] = df_mensal['Coluna'].astype(str)
df_mensal.rename(columns={'Coluna': 'Mês'}, inplace=True)
df_mensal.reset_index(drop=True, inplace=True)

In [33]:
df_mensal.to_csv('/content/drive/MyDrive/data/Agregado/Dados_Agregados_Mensal.csv', index=False)